In [ ]:
import requests
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [ ]:
# função para aplicar o paralelismo
def mapp(fn,data, workers=8):
  tasks=[]
  with ThreadPoolExecutor(max_workers=workers) as tex:
    tasks=[tex.submit(fn,i) for i in data]
  for task in as_completed(tasks):
    print(task.result())

In [ ]:
def downloader(url):
    dirpath="downloads/"
    try:
        r = requests.get(url, stream=True)
        total = int(r.headers.get('content-length', 0))
        if "Content-Disposition" in r.headers.keys():
            fname = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
        else:
            fname = url.split("/")[-1]
        with open(dirpath+fname, 'wb') as file, tqdm(desc=fname, total=total, unit='iB', unit_scale=True, unit_divisor=1024) as bar:
            for data in r.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)
    finally:
        return str(r.status_code) + " " + url

In [ ]:
# mapp(downloader,urllist)